In [3]:
import model
from data_loading import get_val_transforms, WeatherDataModule
import torch
import numpy as np

model1 = model.get_base_model("resnet50", 11)
state_dict = torch.load("models/resnet50-val_acc=0.91.ckpt")['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("model.", "")] = state_dict.pop(key)
model1.load_state_dict(state_dict,strict=False)
model2 = model.get_base_model("efficientnetb0", 11)
state_dict = torch.load("models/efficientnetb0-val_acc=0.93.ckpt")['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("model.", "")] = state_dict.pop(key)
model2.load_state_dict(state_dict,strict = False)
model3 = model.get_base_model("efficientnetb1", 11)
state_dict = torch.load("models/efficientnetb1-val_acc=0.93.ckpt")['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("model.", "")] = state_dict.pop(key)
model3.load_state_dict(state_dict,strict=False)
model4 = model.get_base_model("mobilenet", 11)
state_dict = torch.load("models/mobilenet-val_acc=0.90.ckpt")['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("model.", "")] = state_dict.pop(key)
model4.load_state_dict(state_dict,strict=False)
model5 = model.get_base_model("swin", 11)
state_dict = torch.load("models/swin-val_acc=0.94.ckpt")['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("model.", "")] = state_dict.pop(key)
model5.load_state_dict(state_dict,strict=False)

models = [model1, model2, model3, model4, model5]
model_names = ["resnet50", "efficientnetb0", "efficientnetb1", "mobilenet", "swin"]

/tmp/ipykernel_32367/2209584414.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("models/resnet50-val_acc=0.91.ckpt")['state_dict']
/tmp/ipykernel

In [4]:
datamodule = WeatherDataModule("./data/weather-dataset", 32, 1, get_val_transforms(), get_val_transforms())
datamodule.setup()
val_dataloader = datamodule.val_dataloader()
X_val = []
y_val = []
for images, labels in val_dataloader:
    X_val.append(images)
    y_val.append(labels)
X_val = torch.cat(X_val)
y_val = torch.cat(y_val)

In [6]:
predictions = []
X_val = X_val.to("cuda")
for model in models:
    print(f"Running model {model_names[models.index(model)]}")
    model.to("cuda")
    model.eval()
    with torch.no_grad():
        predictions.append(model(X_val))
predictions = torch.stack(predictions)
final_predictions = torch.mode(predictions, dim=0).values
final_predictions = torch.argmax(final_predictions, dim=1)
y_val = y_val.to("cuda")
accuracy = (final_predictions == y_val).float().mean().item()
print(f"Ensemble accuracy max vote: {accuracy:.4f}")
predictions = predictions.mean(dim=0)
final_predictions = torch.argmax(predictions, dim=1)
accuracy = (final_predictions == y_val).float().mean().item()
print(f"Ensemble accuracy mean vote: {accuracy:.4f}")

Running model resnet50
Running model efficientnetb0
Running model efficientnetb1
Running model mobilenet
Running model swin
Ensemble accuracy max vote: 0.9417
Ensemble accuracy mean vote: 0.9397
